In [0]:
%python 
df_mac_23 = spark.sql(f''' 
    select mac_addr, mac_addr_hashed, 
           case when mac_addr = 'F8:01:B4:34:76:07' then '1'
                when mac_addr = '64:E4:A5:18:40:30' then '2'
                when mac_addr = '7C:64:6C:F3:CA:E0' then '4'
                when mac_addr = 'F8:01:B4:09:47:C1' then '5' else '89' end as mac_seq
    from   eic_data_private.tlamp.rbt_mac_addr_webos23
    where  mac_addr in ('F8:01:B4:34:76:07', 
                        '64:E4:A5:18:40:30', 
                        '7C:64:6C:F3:CA:E0', 
                        'F8:01:B4:09:47:C1')
''')

str_mac_list_23 = ''
for row in df_mac_23.collect() :
    if str_mac_list_23 == '':
        str_mac_list_23 = f"'{row.mac_addr_hashed}'"
    else:
        str_mac_list_23 = str_mac_list_23 + f",'{row.mac_addr_hashed}'"
print(str_mac_list_23)

In [0]:
%python 
df_mac_24 = spark.sql(f''' 
    select mac_addr,
           mac_addr_hashed, 
           case when mac_addr = '00:A1:59:D9:5D:E0' then '3'
                when mac_addr = '00:A1:59:56:14:EF' then '6'
                when mac_addr = '00:A1:59:B6:52:D6' then '7' else '99' end as mac_seq
    from   eic_data_private.tlamp.rbt_mac_addr_webos24
    where  mac_addr in ('00:A1:59:D9:5D:E0', 
                        '00:A1:59:56:14:EF', 
                        '00:A1:59:B6:52:D6')
''')

str_mac_list_24 = ''
for row in df_mac_24.collect() :
    if str_mac_list_24 == '':
        str_mac_list_24 = f"'{row.mac_addr_hashed}'"
    else:
        str_mac_list_24 = str_mac_list_24 + f",'{row.mac_addr_hashed}'"
print(str_mac_list_24)

In [0]:
%python
list_cc = ['AT', 'FR']

df = None
for cc in list_cc:
    df_t_23 = spark.sql(f''' 
            SELECT
                X_device_platform as `platform_code`,  -- 필수 열
                X_device_product as `platform_version`,-- 필수 열
                X_Device_Country as `country_code`,    -- 필수/조건 열
                log_create_time,                       -- 필수/조건 열
                DEVICE_NAME as soc,
                mac_addr, 
                x_device_sales_model as `sales_model_code`, -- 필수 열
                context_name,                          -- 필수/조건 열  
                message_id,                            -- 필수/조건 열
                normal_log:top_10
            FROM eic_data_ods.tlamp_private.normal_log_webos23
            WHERE 1=1
                AND X_device_country = '{cc}' 
                AND context_name = 'MM'
                AND message_id = 'NL_MM_PROCLIST'
                AND date_ym >= '2025-04'
                AND date_ym <= '2025-08'
                AND mac_addr in ({str_mac_list_23})
        ''')
    
    df_t_24 = spark.sql(f''' 
            SELECT
                X_device_platform as `platform_code`,  -- 필수 열
                X_device_product as `platform_version`,-- 필수 열
                X_Device_Country as `country_code`,    -- 필수/조건 열
                log_create_time,                       -- 필수/조건 열
                DEVICE_NAME as soc,
                mac_addr, 
                x_device_sales_model as `sales_model_code`, -- 필수 열
                context_name,                          -- 필수/조건 열  
                message_id,                            -- 필수/조건 열
                normal_log:top_10
            FROM eic_data_ods.tlamp_private.normal_log_webos24
            WHERE 1=1
                AND X_device_country = '{cc}' 
                AND context_name = 'MM'
                AND message_id = 'NL_MM_PROCLIST'
                AND date_ym >= '2025-04'
                AND date_ym <= '2025-08'
                AND mac_addr in ({str_mac_list_24})
        ''')
    
    print(cc, df_t_23.count(), df_t_24.count())
    
    df_t = df_t_23.union(df_t_24)
    
    if df == None :
        df = df_t
    else:
        df = df.union(df_t)

    df_t = None
    df_t_23 = None 
    df_t_24 = None

In [0]:
df_mac = df_mac_23.union(df_mac_24)

df_result = df.join(
                df_mac,         
                df["mac_addr"] == df_mac["mac_addr_hashed"],
                how="left")\
              .select(
                  # df_mac["mac_addr"],
                  df_mac["mac_seq"],
                  df["platform_code"],
                  df["platform_version"],
                  df["soc"],
                  df["country_code"],
                  df["log_create_time"],
                  df["sales_model_code"],
                  df["context_name"],
                  df["message_id"],
                  df["top_10"],
              )

display(df_result)